In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# 데이터 로드
df = pd.read_csv('data/ratings.csv')
df.head(2)

In [ ]:
# 사용자와 영화를 각각 인덱싱합니다.
user_ids = df["userId"].unique().tolist()
user2user_encoded = {x: i for i, x in enumerate(user_ids)}
userencoded2user = {i: x for i, x in enumerate(user_ids)}

movie_ids = df["movieId"].unique().tolist()
movie2movie_encoded = {x: i for i, x in enumerate(movie_ids)}
movie_encoded2movie = {i: x for i, x in enumerate(movie_ids)}

# 인덱스를 적용합니다.
df["user"] = df["userId"].map(user2user_encoded)
df["movie"] = df["movieId"].map(movie2movie_encoded)

# 데이터를 훈련 세트와 테스트 세트로 분리합니다.
train, test = train_test_split(df, test_size=0.2)

In [ ]:
# 모델 작성
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate
from tensorflow.keras.models import Model

# 임베딩의 차원
embedding_dim = 50

# 사용자와 영화 각각에 대한 입력 레이어를 정의합니다.
user_id_input = Input(shape=(1,), name='user')
movie_id_input = Input(shape=(1,), name='movie')

# 각 입력에 대해 임베딩 레이어를 만듭니다.
user_embedding = Embedding(output_dim=embedding_dim, 
                           input_dim=len(user_ids)+1, 
                           input_length=1, 
                           name='user_embedding')(user_id_input)
movie_embedding = Embedding(output_dim=embedding_dim, 
                            input_dim=len(movie_ids)+1, 
                            input_length=1, 
                            name='movie_embedding')(movie_id_input)

# 임베딩 벡터를 펼칩니다.
user_vector = Flatten()(user_embedding)
movie_vector = Flatten()(movie_embedding)

# 두 벡터를 합쳐서 하나의 벡터를 만듭니다.
concat = Concatenate()([user_vector, movie_vector])

# 완전연결 계층을 추가합니다.
dense1 = Dense(128, activation='relu')(concat)
dense2 = Dense(32, activation='relu')(dense1)
output = Dense(1)(dense2)

# 모델을 만듭니다.
model = Model(inputs=[user_id_input, movie_id_input], outputs=output)
model.compile(loss='mse', optimizer='adam')

# 모델 훈련
history = model.fit([train.user.values, train.movie.values], 
                    train.rating.values,
                    batch_size=64, 
                    epochs=5, 
                    validation_split=0.2)

In [ ]:
pd.DataFrame(history.history).plot()